In [1]:
from datetime import datetime

import numpy as np

import os

import pandas as pd

import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model

from time import time, gmtime, strftime

from bert.tokenization.bert_tokenization import FullTokenizer

from utils.models.modelUtils import loadMeanFromFile, calcAccConcatModel

from final_models import create_image_inceptionv3_model, create_text_model, create_model_meta, buildConcatModelTitleCommentsMetaVisual

from utils.callbacks.MyCallbacks import MyCallbacks
from utils.datagenUtils.four_modal.DataSeqFourModels import DataSequenceFourModels
from utils.datagenUtils.datagenUtils import checkDataframe

from utils.telegramUtils.telegram_bot import telegram_send_message
from utils.textUtils.textpreprocessing import FakeDetectionDataTrainVal, FakeDetectionDataTest
from utils.textUtils.commentsProcessing import FakeDetectionDataCommentsTest, FakeDetectionDataCommentsTrainVal

from utils.callbacks.callbackUtils import plotTimesPerEpoch

from utils.fileDirUtils.fileDirUtils import createDirIfNotExists

from sklearn.metrics import accuracy_score


In [2]:
#Verbose settings:
verbose = False
TF_VERBOSE = 1 # 1 = Progress bar 2 = one line per epoch only!
TF_DETERMINISTIC_OPS = 1 # Makes everything also on GPU deterministic

# Classes:
NUM_CLASS = 2  # FAKE | NO FAKE

# Hyperparameters
GLOBAL_BATCH_SIZE = 64
EPOCHS = 20

# Optimizer parameters:
# Adam
LEARNING_RATE = 1e-5
BETA_1 = 0.9
BETA_2 = 0.999
EPSILON = 1e-8

#optimizers:

optimizer = Adam(LEARNING_RATE, BETA_1, BETA_2, EPSILON)

# Bert Parameters
MAX_SEQUENCE_LENGTH = 128 # from model definition!

# Image Model  Parameters
IMG_WIDTH = 768
IMG_HEIGHT = 768
IMG_DEPTH = 3
IMG_SIZES = (IMG_WIDTH, IMG_HEIGHT)

# Training switches
RETRAIN_WHOLE_MODEL = False
loadPretrained = True # Attention that model weights path is set !

# Custom telegram send text 
CUSTOM_TEXT = f'Batch Size: {GLOBAL_BATCH_SIZE}, Epochs: {EPOCHS}, Optimizer: Adam, Learning Rate; {LEARNING_RATE}, Beta_1: {BETA_1}, Beta_2: {BETA_2}, Epsilon: {EPSILON}, Image Sizes: {IMG_SIZES}'


telegram_send_message(f'-----------------START-----------------')
print('START')
print(CUSTOM_TEXT)
telegram_send_message(CUSTOM_TEXT)

START
Batch Size: 64, Epochs: 20, Optimizer: Adam, Learning Rate; 1e-05, Beta_1: 0.9, Beta_2: 0.999, Epsilon: 1e-08, Image Sizes: (768, 768)


In [3]:
pathToRootModelDir = '/home/armin/repos/fkd-model-handling/models/best_models'

pathToBertModelTitle = os.path.join(pathToRootModelDir, 'single_text_title', 'weights-improvement-02-0.88.hdf5')
pathToBertModelComments = os.path.join(pathToRootModelDir, 'single_text_comments', 'weights-improvement-03-0.87.hdf5')
pathToImageModel = os.path.join(pathToRootModelDir, 'single_visual', 'weights-improvement-02-0.81.hdf5')
pathToMetaModel = os.path.join(pathToRootModelDir, 'single_meta', 'weights-improvement-100-0.62.hdf5')

pathToImagesTrain = '/home/armin/repos/FKD-Dataset/006_images_resized_2/train/' 
pathToCSVWithFileNamesAndLabelsTrain = '/home/armin/repos/FKD-Dataset/008_text_image_meta_label/train_text_image_meta_label.csv'

pathToImagesVal = '/home/armin/repos/FKD-Dataset/006_images_resized_2/val/' 
pathToCSVWithFileNamesAndLabelsVal = '/home/armin/repos/FKD-Dataset/008_text_image_meta_label/val_text_image_meta_label.csv'

pathToImagesTest = '/home/armin/repos/FKD-Dataset/006_images_resized_2/test/' 
pathToCSVWithFileNamesAndLabelsTest = '/home/armin/repos/FKD-Dataset/008_text_image_meta_label/test_text_image_meta_label.csv'

checkpointDir = '/home/armin/repos/FKD-Dataset/011_checkpoints'

pathToMeans = '/home/armin/repos/FKD-Dataset/010_configs/means_resized_768.txt'

titleCommentsVisualMetaModelWeightsPath = "/home/armin/repos/FKD-Dataset/011_checkpoints/model_concat_title_comments_2020-07-18_20:37/weights-improvement-05-0.94.hdf5"

root = '/home/armin/repos/fkd-model-handling/'
bert_model_dir = os.path.join(root, 'multi_cased_L-12_H-768_A-12')
bert_config_file = os.path.join(bert_model_dir, "bert_config.json")
bert_ckpt_file = os.path.join(bert_model_dir, "bert_model.ckpt") 

In [4]:
# Other ettings

# Time settings:
current_time = datetime.now().strftime("%Y-%m-%d_%H:%M")

#Checkpoint settings:
checkpoint_name = "model_concat_title_comments_visual_meta"

checkpointDir = os.path.join(checkpointDir, (checkpoint_name + '_' + current_time))

fileName="weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5"
filePath = os.path.join(checkpointDir, fileName)

In [5]:
# Callback Handling:
tensorboardDir = os.path.join(checkpointDir, 'tensorboard')

createDirIfNotExists(tensorboardDir)
createDirIfNotExists(checkpointDir)


callbacks_list = MyCallbacks(tensorboardDir, filePath, earlyStopping=True).createCheckpoints()

Directory /home/armin/repos/FKD-Dataset/011_checkpoints/model_concat_title_comments_visual_meta_2020-07-20_06:50/tensorboard does not exist, creating it instead!


In [6]:
df_train_ = pd.read_csv(pathToCSVWithFileNamesAndLabelsTrain, header=0, sep='\t')
df_test_ = pd.read_csv(pathToCSVWithFileNamesAndLabelsTest, header=0, sep='\t')
df_val_ = pd.read_csv(pathToCSVWithFileNamesAndLabelsVal, header=0, sep='\t')

df_train_['2_way_label'] = df_train_['2_way_label'].apply(lambda x: np.array(x))
df_test_['2_way_label'] = df_test_['2_way_label'].apply(lambda x: np.array(x))
df_val_['2_way_label'] = df_val_['2_way_label'].apply(lambda x: np.array(x))

df_train = df_train_
df_test = df_test_
df_val = df_val_

# df_train = df_train[:5000]
# df_test = df_test[:5000]
# df_val = df_val[:5000]

df_train = checkDataframe(df_train, GLOBAL_BATCH_SIZE)
df_val = checkDataframe(df_val, GLOBAL_BATCH_SIZE)
df_test = checkDataframe(df_test, GLOBAL_BATCH_SIZE)


In [7]:
tokenizer = FullTokenizer(vocab_file=os.path.join(bert_model_dir, "vocab.txt"))

all_title_data = FakeDetectionDataTrainVal(df_train, df_val, tokenizer, [0,1], MAX_SEQUENCE_LENGTH)

all_title_test = FakeDetectionDataTest(df_test, tokenizer, [0,1], MAX_SEQUENCE_LENGTH)


comments_data_train = FakeDetectionDataCommentsTrainVal(df_train, df_val, tokenizer, [0,1], MAX_SEQUENCE_LENGTH)

comments_data_test = FakeDetectionDataCommentsTest(df_test, tokenizer, [0,1], MAX_SEQUENCE_LENGTH)



train_title_x = all_title_data.train_x
train_title_val_x = all_title_data.val_x
test_title_x = all_title_test.test_x

comments_train_x = comments_data_train.train_x
comments_val_x = comments_data_train.val_x
comments_test_x = comments_data_test.test_x

560576it [02:10, 4302.18it/s]
58944it [00:13, 4431.78it/s]


max seq_len 687


58944it [00:13, 4425.03it/s]


max seq_len 128


560576it [33:00, 283.08it/s]
58944it [03:34, 274.90it/s]


max seq_len 55475


58944it [03:32, 276.83it/s]


max seq_len 39424


In [8]:
meansOfDataset = loadMeanFromFile(pathToMeans, verbose)

train_seq = DataSequenceFourModels(df_train, pathToImagesTrain, train_title_x, comments_train_x, GLOBAL_BATCH_SIZE, IMG_SIZES, meansOfDataset)

test_seq = DataSequenceFourModels(df_test, pathToImagesTest, test_title_x, comments_test_x, GLOBAL_BATCH_SIZE, IMG_SIZES, meansOfDataset)

val_seq = DataSequenceFourModels(df_val,pathToImagesVal, train_title_val_x, comments_val_x, GLOBAL_BATCH_SIZE, IMG_SIZES, meansOfDataset)

STEP_SIZE_TRAIN = len(df_train) // GLOBAL_BATCH_SIZE
STEP_SIZE_TEST = len(df_test) // GLOBAL_BATCH_SIZE
STEP_SIZE_VAL = len(df_val) // GLOBAL_BATCH_SIZE

In [9]:
# from tensorflow.io.gfile import GFile

# from tensorflow.keras.applications.inception_v3 import InceptionV3
# from tensorflow.keras.applications.resnet_v2 import ResNet50V2
# from tensorflow.keras.applications.resnet_v2 import ResNet101V2

# from tensorflow.keras.layers import Concatenate, Dense, Dropout, GlobalAveragePooling2D, Input, Lambda
# from tensorflow.keras import Model

# from tensorflow.keras.initializers import GlorotNormal
# from tensorflow.keras.regularizers import l1_l2



# def buildConcatModelTitleCommentsMetaVisual(model_bert_title, model_bert_comments, model_image, model_meta, NUM_CLASS):
#     model_bert_title_output = model_bert_title.get_layer('bert_output_layer_768_title').output
#     model_bert_comments_output = model_bert_comments.get_layer('bert_output_layer_768').output
#     model_meta_output = model_meta.get_layer('final_output').output
#     model_image_output = model_image.get_layer('img_dense_768').output
    
#     # Build new models, without softmax
#     model_bert_title = Model(model_bert_title.inputs, model_bert_title_output)    
#     model_bert_comments = Model(model_bert_comments.inputs, model_bert_comments_output)
#     model_image = Model(model_image.inputs, model_image_output)    
#     model_meta = Model(model_meta.inputs, model_meta_output)
    
#     # Build multimodal model
    
#     concatenate = Concatenate()([model_bert_title.output, model_bert_comments.output, model_image_output]) # Fusion
#     x = Dense(1024, activation = 'relu')(concatenate)
#     x = Dropout(0.4)(x)
#     x = Dense(6, activation = 'relu')(x)
#     concatenateMeta = Concatenate()([model_meta.output, x]) # Fusion
#     x = Dropout(0.4)(concatenateMeta)
   
#     output = Dense(NUM_CLASS, activation='softmax', name='final_multimodal_output')(x)
#     model_concat = Model([model_image.input, model_bert_title.input, model_bert_comments.input, model_meta.input], output)
#     return model_concat

In [10]:
mirrored_strategy = tf.distribute.MirroredStrategy()
with mirrored_strategy.scope():
    # Preparing:
    model_bert_title, model_bert_title_custom_layer = create_text_model(max_seq_len=MAX_SEQUENCE_LENGTH, bert_ckpt_file=bert_ckpt_file, bert_config_file= bert_config_file,NUM_CLASS=NUM_CLASS, overwriteLayerAndEmbeddingSize=False, isPreTrained=True,  pathToBertModelWeights=pathToBertModelTitle, isTrainable=RETRAIN_WHOLE_MODEL)
    model_bert_comments, model_bert_comments_custom_layer = create_text_model(max_seq_len=MAX_SEQUENCE_LENGTH, bert_ckpt_file=bert_ckpt_file, bert_config_file= bert_config_file,NUM_CLASS=NUM_CLASS, overwriteLayerAndEmbeddingSize=False, isPreTrained=True,  pathToBertModelWeights=pathToBertModelComments, isTrainable=RETRAIN_WHOLE_MODEL) 
    model_image, model_image_custom_layer = create_image_inceptionv3_model(NUM_CLASS=NUM_CLASS,isPreTrained=True,pathToInceptionV3ModelWeights=pathToImageModel, isTrainable=RETRAIN_WHOLE_MODEL)
    model_meta, model_meta_custom_layer = create_model_meta(NUM_CLASS, 2, True, pathToMetaModel, isTrainable=RETRAIN_WHOLE_MODEL)
    
    
    
    # Handling same layer name error:
    for i, layer in enumerate(model_bert_title.layers):
        layer._name = layer._name + '_title' # Consider the _ for the setter
    
    model_concat = buildConcatModelTitleCommentsMetaVisual(model_bert_title, model_bert_comments, model_image, model_meta, 2)
    
    if loadPretrained:
        model_concat.load_weights(titleCommentsVisualMetaModelWeightsPath)

    if verbose:
        print('--------------Text Title MODEL --------------')
        model_bert_title.summary()
        print('--------------Text Comments MODEL --------------')
        model_bert_comments.summary()
        print('--------------CONCAT MODEL --------------')
        model_concat.summary()
        for l in model_concat.layers:
            print(l.name, l.trainable)
    
    
    model_concat.compile(loss=keras.losses.SparseCategoricalCrossentropy(),
          optimizer=optimizer,
          metrics=['accuracy'])
    
    history = model_concat.fit(train_seq,
                    steps_per_epoch=STEP_SIZE_TRAIN,epochs=EPOCHS, 
                    validation_data=val_seq,
                    validation_steps=STEP_SIZE_VAL, 
                    callbacks=callbacks_list,
                    use_multiprocessing=False,
                    verbose=TF_VERBOSE,
                    workers=12
                   )

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')
bert shape (None, 128, 768)
bert shape (None, 128, 768)
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).

KeyboardInterrupt: 

In [ ]:
# for i, layer in enumerate(model_bert_title.layers):
#     layer._name = layer._name + '_title' # Consider the _ for the setter
        
# for i, layer in enumerate(model_bert_title.layers):
#     print(layer.name)

In [11]:
%%capture

plot_model(model_concat, to_file=os.path.join(checkpointDir,'multiple_inputs_image_text_meta_pretrained.png'), show_shapes=True, dpi=600, expand_nested=False)
plot_model(model_concat, to_file=os.path.join(checkpointDir,'multiple_inputs_image_text_meta_pretrained_nested.png'), show_shapes=True, dpi=600, expand_nested=True)

In [12]:
# # For later model loading
# modelFile = '/home/armin/repos/FKD-Dataset/011_checkpoints/model_concat_title_comments_visual_meta_2020-07-19_14:04/weights-improvement-04-0.95.hdf5'

# model_concat.load_weights(modelFile)

In [13]:
plotTimesPerEpoch(callbacks_list)

It took so long to train on one epoch: [] minutes


In [14]:
calcAccConcatModel(model_concat, val_seq, GLOBAL_BATCH_SIZE, 'Val')
calcAccConcatModel(model_concat, test_seq, GLOBAL_BATCH_SIZE, 'Test')

131/921 [===>..........................] - ETA: 39:53

/home/armin/repos/fkd-model-handling/utils/datagenUtils/four_modal/DataSeqFourModels.py:67: RuntimeWarning: divide by zero encountered in true_divide
  pixels = (pixels - self.meansOfDataset) / stds # Applying normalization


921/921 [==============================] - 2829s 3s/step
Val Accuracy is 0.9457281487513572
921/921 [==============================] - 2829s 3s/step
Test Accuracy is 0.9463728284473398


In [15]:
telegram_send_message(f'-----------------DONE-----------------')
print('done')

done
